In [4]:
import re
import pymongo
from pymongo import MongoClient
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [5]:
## get data from mongodb
client = MongoClient()
db=client.nh4
ids=[]
addrs=[]
for issue in db.issues.find():
    ids.append(issue["id"])
    addrs.append(issue["address"])

In [6]:
r = re.compile('^\d+(-|/)?\d*\s.*(\
(\sst\.?($|\s|(reet($|\s))))\
|(\save?\.?($|\s|(enue($|\s))))\
|(\sr((d\.?($|\s))|(oad($|\s))))\
|(\sdr\.?($|\s|(ive($|\s))))\
|(\spl\.?($|\s|(ace($|\s))))\
|(\sterrace)\
|(\sb((lvd\.?($|\s))|(oulivard($|\s))))\
|(\spkwy\.?|parkway)\
)')

o = re.compile("(\
(\sst\.?($|\s|(reet($|\s))))\
|(\save?\.?($|\s|(enue($|\s))))\
|(\sr((d\.?($|\s))|(oad($|\s))))\
|(\sdr\.?($|\s|(ive($|\s))))\
|(\spl\.?($|\s|(ace($|\s))))\
|(\sterrace)\
|(\sb((lvd\.?($|\s))|(oulivard($|\s))))\
|(\spkwy\.?|parkway)\
)")

num = re.compile("^\d+\s*(-|/)?\s*\d*\s")

ll = re.compile('\(\d+\.')

In [7]:
## function to replace the street type with the correct format...
def clean_street_type(street_type):
    street_type=street_type.strip(" ")
    street_type_dict={
        "av.":" avenue",
        "ave.":"avenue",
        "ave":" avenue",
        "av":" avenue",
        "st":" street",
        "st.": " street",
        "rd":" road",
        "rd.":" road",
        "tr":" terrace",
        "dr":" drive",
        "dr.":" drive",
        "blvd.":"boulivard",
        "blvd":" boulivard",
        "pl": " place",
        "pl.": " place"
    }
    if street_type in street_type_dict:
        street_type=street_type_dict[street_type]
    else:
        street_type = " "+street_type
    return street_type

In [8]:
## here, we extract street names from address
street_names=[]

for addr in addrs:
    street_name=''
    addr_formatted = addr.lower().split()
    if len(addr_formatted)>0:
        ##check first if address is in the regular format...
        if re.match(r,addr.lower()):
            street=re.match(r,addr.lower()).group(0)
            number=re.match(num,addr.lower()).group(0)
            street_type=re.search(o,addr.lower()).group(0)
            street_name=street.replace(number,"").replace(street_type,clean_street_type(street_type))
    street_names.append(street_name)

In [12]:
for i in range(len(ids)):
    db.issues.update_one({"id":ids[i]},{"$set" :{"street_name":street_names[i]}})
    